In [3]:
###import####
import json
import pandas as pd
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import seaborn as sns

%matplotlib inline
import math
import json
import pandas as pd
from pandas import json_normalize
import numpy as np

import requests
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

from matplotlib import cm
import matplotlib.pyplot as plt
import cmasher as cmr
import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import matplotlib.cm
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.projections import get_projection_class
from matplotlib.patches import Arc
import io
import matplotlib.image as image
from matplotlib import transforms
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch

# Import StatsBomb Data - FAWSL Season 2019/2020

In [4]:
####get StatsBomb Data########
with open (r"YOUR FILEPATH\open-data-master\data\\matches\37\42.json", "r") as f:

                obj = json.load(f)




mlist = []

objlist = []

dflist = []

for match in obj:

                mid = match["match_id"]

                mlist.append(mid)




mlist = [str(m) for m in mlist]

mlist = [m+".json" for m in mlist]




for n in mlist:

                file = r"YOUR FILEPATH\open-data-master\data\/events/" + "/" + n

                with open (file, "r", encoding="utf-8") as f:

                                obj = json.load(f)

                                objlist.append(obj)

for obj in objlist:

    df = json_normalize(obj)

    dflist.append(df)

result = pd.concat(dflist, sort=True)
result
###drop unrelevant events#####
result1 = result[(result["type.name"]=='Starting XI')| (result["type.name"]=='Half Start')| (result["type.name"]=='Tactical Shift')| (result["type.name"]=='Injury Stoppage')|(result["type.name"]=='Half End')|(result["type.name"]=='Substitution')]
index_list = result1.index.tolist()
result= result.drop(index_list)

result = result.reset_index()
result

C:\Users\philipp.hollenhorst\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,level_0,50_50.outcome.id,50_50.outcome.name,bad_behaviour.card.id,bad_behaviour.card.name,ball_receipt.outcome.id,ball_receipt.outcome.name,ball_recovery.offensive,ball_recovery.recovery_failure,block.deflection,block.offensive,block.save_block,carry.end_location,clearance.aerial_won,clearance.body_part.id,clearance.body_part.name,clearance.head,clearance.left_foot,clearance.other,clearance.right_foot,counterpress,dribble.no_touch,dribble.nutmeg,dribble.outcome.id,dribble.outcome.name,dribble.overrun,duel.outcome.id,duel.outcome.name,duel.type.id,duel.type.name,duration,foul_committed.advantage,foul_committed.card.id,foul_committed.card.name,foul_committed.offensive,foul_committed.penalty,foul_committed.type.id,foul_committed.type.name,foul_won.advantage,foul_won.defensive,foul_won.penalty,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.end_location,goalkeeper.lost_in_play,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.punched_out,goalkeeper.saved_to_post,goalkeeper.shot_saved_off_target,goalkeeper.shot_saved_to_post,goalkeeper.success_in_play,goalkeeper.technique.id,goalkeeper.technique.name,goalkeeper.type.id,goalkeeper.type.name,half_end.early_video_end,half_start.late_video_start,id,index,injury_stoppage.in_chain,interception.outcome.id,interception.outcome.name,location,minute,miscontrol.aerial_won,off_camera,out,pass.aerial_won,pass.angle,pass.assisted_shot_id,pass.body_part.id,pass.body_part.name,pass.cross,pass.cut_back,pass.deflected,pass.end_location,pass.goal_assist,pass.height.id,pass.height.name,pass.inswinging,pass.length,pass.miscommunication,pass.no_touch,pass.outcome.id,pass.outcome.name,pass.outswinging,pass.recipient.id,pass.recipient.name,pass.shot_assist,pass.straight,pass.switch,pass.technique.id,pass.technique.name,pass.through_ball,pass.type.id,pass.type.name,period,play_pattern.id,play_pattern.name,player.id,player.name,position.id,position.name,possession,possession_team.id,possession_team.name,related_events,second,shot.aerial_won,shot.body_part.id,shot.body_part.name,shot.deflected,shot.end_location,shot.first_time,shot.follows_dribble,shot.freeze_frame,shot.key_pass_id,shot.one_on_one,shot.open_goal,shot.outcome.id,shot.outcome.name,shot.redirect,shot.saved_off_target,shot.saved_to_post,shot.statsbomb_xg,shot.technique.id,shot.technique.name,shot.type.id,shot.type.name,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,tactics.formation,tactics.lineup,team.id,team.name,timestamp,type.id,type.name,under_pressure
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.771676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c0cc1e3b-af5c-448c-82cc-08e546a72f5b,5,NaN,NaN,NaN,"[61.0, 40.1]",0,NaN,NaN,NaN,NaN,2.900027,NaN,40.0,Right Foot,NaN,NaN,NaN,"[54.1, 41.8]",NaN,1.0,Ground Pass,NaN,7.106335,NaN,NaN,NaN,NaN,NaN,10251.0,Fara Williams,NaN,NaN,NaN,NaN,NaN,NaN,65.0,Kick Off,1,9,From Kick Off,10190.0,Jade Moore,10.0,Center Defensive Midfield,2,974,Reading WFC,[99c4a406-f3d4-4bd0-b5b0-3a0598ae54dd],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,974,Reading WFC,00:00:00.046,30,Pass,NaN
1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99c4a406-f3d4-4bd0-b5b0-3a0598ae54dd,6,NaN,NaN,NaN,"[54.1, 41.8]",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,From Kick Off,10251.0,Fara Williams,19.0,Center Attacking Midfield,2,974,Reading WFC,[c0cc1e3b-af5c-448c-82cc-08e546a72f5b],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Clean and manipulate the data

In [5]:
###data cleaning and manipulation#####

###combine pass and carry end location####
result = result.reset_index(drop=True)
result["end_location"] = result["pass.end_location"]
result["carryendl"] = result["carry.end_location"]
result["end_location"].update(result.pop("carryendl"))

###extract x and y locations######
location = pd.DataFrame(result[(result["type.name"]=="Pass")| (result["type.name"]=="Carry")]["location"].values.tolist(),columns=["xstart", "ystart"])
end_location = pd.DataFrame(result[(result["type.name"]=="Pass")| (result["type.name"]=="Carry")]["end_location"].values.tolist(),columns=["xend", "yend"])

result0 = pd.concat([location, end_location], axis=1)

### create new columns to evaluate behavior under pressure####
result['underpressure'] = result['under_pressure'].apply(lambda x: 1 if x ==True else 0)
result['nextactionu_p'] = result['underpressure'].shift(-1)
result['nextaction_name'] = result['type.name'].shift(-1)
result['nextaction_name2'] = result['type.name'].shift(-2)

result["prior_action_name"] = result['type.name'].shift(1)
result["prior_action_name2"] = result['type.name'].shift(2)

result = result[(result["type.name"]=="Pass") | (result["type.name"]=="Carry")]
result = result.reset_index(drop=True)
result = pd.concat([result,result0],axis=1)
result

,level_0,50_50.outcome.id,50_50.outcome.name,bad_behaviour.card.id,bad_behaviour.card.name,ball_receipt.outcome.id,ball_receipt.outcome.name,ball_recovery.offensive,ball_recovery.recovery_failure,block.deflection,block.offensive,block.save_block,carry.end_location,clearance.aerial_won,clearance.body_part.id,clearance.body_part.name,clearance.head,clearance.left_foot,clearance.other,clearance.right_foot,counterpress,dribble.no_touch,dribble.nutmeg,dribble.outcome.id,dribble.outcome.name,dribble.overrun,duel.outcome.id,duel.outcome.name,duel.type.id,duel.type.name,duration,foul_committed.advantage,foul_committed.card.id,foul_committed.card.name,foul_committed.offensive,foul_committed.penalty,foul_committed.type.id,foul_committed.type.name,foul_won.advantage,foul_won.defensive,foul_won.penalty,goalkeeper.body_part.id,goalkeeper.body_part.name,goalkeeper.end_location,goalkeeper.lost_in_play,goalkeeper.outcome.id,goalkeeper.outcome.name,goalkeeper.position.id,goalkeeper.position.name,goalkeeper.punched_out,goalkeeper.saved_to_post,goalkeeper.shot_saved_off_target,goalkeeper.shot_saved_to_post,goalkeeper.success_in_play,goalkeeper.technique.id,goalkeeper.technique.name,goalkeeper.type.id,goalkeeper.type.name,half_end.early_video_end,half_start.late_video_start,id,index,injury_stoppage.in_chain,interception.outcome.id,interception.outcome.name,location,minute,miscontrol.aerial_won,off_camera,out,pass.aerial_won,pass.angle,pass.assisted_shot_id,pass.body_part.id,pass.body_part.name,pass.cross,pass.cut_back,pass.deflected,pass.end_location,pass.goal_assist,pass.height.id,pass.height.name,pass.inswinging,pass.length,pass.miscommunication,pass.no_touch,pass.outcome.id,pass.outcome.name,pass.outswinging,pass.recipient.id,pass.recipient.name,pass.shot_assist,pass.straight,pass.switch,pass.technique.id,pass.technique.name,pass.through_ball,pass.type.id,pass.type.name,period,play_pattern.id,play_pattern.name,player.id,player.name,position.id,position.name,possession,possession_team.id,possession_team.name,related_events,second,shot.aerial_won,shot.body_part.id,shot.body_part.name,shot.deflected,shot.end_location,shot.first_time,shot.follows_dribble,shot.freeze_frame,shot.key_pass_id,shot.one_on_one,shot.open_goal,shot.outcome.id,shot.outcome.name,shot.redirect,shot.saved_off_target,shot.saved_to_post,shot.statsbomb_xg,shot.technique.id,shot.technique.name,shot.type.id,shot.type.name,substitution.outcome.id,substitution.outcome.name,substitution.replacement.id,substitution.replacement.name,tactics.formation,tactics.lineup,team.id,team.name,timestamp,type.id,type.name,under_pressure,end_location,underpressure,nextactionu_p,nextaction_name,nextaction_name2,prior_action_name,prior_action_name2,xstart,ystart,xend,yend
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.771676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c0cc1e3b-af5c-448c-82cc-08e546a72f5b,5,NaN,NaN,NaN,"[61.0, 40.1]",0,NaN,NaN,NaN,NaN,2.900027,NaN,40.0,Right Foot,NaN,NaN,NaN,"[54.1, 41.8]",NaN,1.0,Ground Pass,NaN,7.106335,NaN,NaN,NaN,NaN,NaN,10251.0,Fara Williams,NaN,NaN,NaN,NaN,NaN,NaN,65.0,Kick Off,1,9,From Kick Off,10190.0,Jade Moore,10.0,Center Defensive Midfield,2,974,Reading WFC,[99c4a406-f3d4-4bd0-b5b0-3a0598ae54dd],0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,974,Reading WFC,00:00:00.046,30,Pass,NaN,"[54.1, 41.8]",0,0.0,Ball Receipt*,Carry,NaN,NaN,61.0,40.1,54.1,41.8
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[54.8, 40.5]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2c920858-7fc1-4a61-a064-d436e3cb1212,7,NaN,NaN,NaN,"[54.1, 41.8]",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# get final df and create new columns

In [6]:
import math

###only show relevant columns###
df = result
df = df[df["underpressure"]==1][["type.name","under_pressure","xstart","ystart","xend","yend","underpressure","nextactionu_p","nextaction_name","nextaction_name2","prior_action_name","prior_action_name2","ball_receipt.outcome.name","duration","pass.angle","pass.height.name","pass.length","pass.outcome.name","pass.recipient.name","pass.type.name","play_pattern.name","player.name","position.name","possession_team.name"]]

### create more new columns to evaluate behavior under pressure####
df['beginning'] = np.sqrt(np.square(120-df['xstart']) + np.square(40 - df['ystart']))
df['end'] = np.sqrt(np.square(120 - df['xend']) + np.square(40 - df['yend']))
df["distance"] = (df["end"] - df["beginning"])
df["prog"] = df["end"]/df["beginning"]
df["changex"] = df["xstart"] - df["xend"]
df['angle1'] = np.degrees(np.arctan2(df['yend']-df['ystart'],df['xend']-df['xstart']))
df['angle1'] = np.where(df['angle1']<0,df['angle1']+360,df['angle1'])
df = df.reset_index()
df


,index,type.name,under_pressure,xstart,ystart,xend,yend,underpressure,nextactionu_p,nextaction_name,nextaction_name2,prior_action_name,prior_action_name2,ball_receipt.outcome.name,duration,pass.angle,pass.height.name,pass.length,pass.outcome.name,pass.recipient.name,pass.type.name,play_pattern.name,player.name,position.name,possession_team.name,beginning,end,distance,prog,changex,angle1
0,3,Carry,True,77.7,75.4,84.6,72.5,1,1.0,Miscontrol,Ball Recovery,Ball Receipt*,Pressure,NaN,0.610300,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Natasha Harding,Right Back,Reading WFC,55.158408,48.056321,-7.102087,0.871242,-6.9,337.203479
1,4,Carry,True,33.2,7.6,33.2,2.8,1,0.0,Pressure,Pass,Ball Recovery,Miscontrol,NaN,4.470062,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Laura Vetterlein,Left Back,Reading WFC,92.649879,94.435587,1.785708,1.019274,0.0,270.000000
2,6,Carry,True,44.3,3.7,43.1,4.3,1,0.0,Dribbled Past,Dribble,Pressure,Pass,NaN,0.543583,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Kate Longhurst,Left Defensive Midfield,Reading WFC,83.953439,84.782663,0.829224,1.009877,1.2,153.434949
3,7,Carry,True,71.2,74.1,77.4,74.1,1,0.0,Pass,Ball Receipt*,Ball Recovery,Pressure,NaN,1.742355,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Fara Williams,Center Attacking Midfield,Reading WFC,59.533604,54.567115,-4.966489,0.916577,-6.2,0.000000
4,10,Carry,True,6.8,31.6,8.7,37.6,1,0.0,Pressure,Pass,Ball Receipt*,Pass,NaN,2.297598,NaN,NaN,NaN,NaN,NaN,NaN,From Free Kick,Anne Moorhouse,Goalkeeper,West Ham United LFC,113.511233,111.325873,-2.185360,0.980748,-1.9,72.428741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28489,97735,Pass,True,65.1,9.1,69.6,8.1,1,0.0,Ball Receipt*,Interception,Pass,Carry,NaN,0.263238,-0.218669,Ground Pass,4.609772,Incomplete,Amalie Vevle Eikeland,NaN,From Throw In,Mayumi Pacheco,Left Back,Arsenal WFC,62.998571,59.647045,-3.351526,0.946800,-4.5,347.471192
28490,97741,Carry,True,13.3,68.2,10.7,76.5,1,1.0,Pass,Ball Receipt*,Ball Recovery,Pressure,NaN,8.097732,NaN,NaN,NaN,NaN,NaN,NaN,Regular Play,Kim Little,Right Center Midfield,Reading WFC,110.363626,115.233415,4.869789,1.044125,2.6,107.393322
28491,97742,Pass,True,10.7,76.5,5.0,77.6,1,0.0,Ball Receipt*,Carry,Carry,Ball Recovery,NaN,1.068645,2.950954,Ground Pass,5.805170,NaN,Leah Williamson,NaN,Regular Play,Kim Little,Right Center Midfield,Reading WFC,115.233415,120.990743,5.757328,1.049962,5.7,169.077195
28492,97745,Carry,True,79.0,23.5,83.3,19.2,1,0.0,Pressure,Carry,Dribble,Dribbled Past,NaN,1.330094,NaN,NaN,NaN,NaN,NaN,NaN,Regular Play,Josanne Potter,Left Center Back,Reading WFC,44.195588,42.184476,-2.011112,0.954495,-4.3,315.000000


In [7]:
###some data manipulation####
dfneg = df[df["distance"]<0]
dfneg["distance"] = dfneg["distance"]*-1
dfneg
dfpos = df[df["distance"]>=0]
df = pd.concat([dfneg,dfpos],axis=0,join="inner")
df = df.sort_values(by="index")
df

C:\Users\philipp.hollenhorst\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,type.name,under_pressure,xstart,ystart,xend,yend,underpressure,nextactionu_p,nextaction_name,nextaction_name2,prior_action_name,prior_action_name2,ball_receipt.outcome.name,duration,pass.angle,pass.height.name,pass.length,pass.outcome.name,pass.recipient.name,pass.type.name,play_pattern.name,player.name,position.name,possession_team.name,beginning,end,distance,prog,changex,angle1
0,3,Carry,True,77.7,75.4,84.6,72.5,1,1.0,Miscontrol,Ball Recovery,Ball Receipt*,Pressure,NaN,0.610300,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Natasha Harding,Right Back,Reading WFC,55.158408,48.056321,7.102087,0.871242,-6.9,337.203479
1,4,Carry,True,33.2,7.6,33.2,2.8,1,0.0,Pressure,Pass,Ball Recovery,Miscontrol,NaN,4.470062,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Laura Vetterlein,Left Back,Reading WFC,92.649879,94.435587,1.785708,1.019274,0.0,270.000000
2,6,Carry,True,44.3,3.7,43.1,4.3,1,0.0,Dribbled Past,Dribble,Pressure,Pass,NaN,0.543583,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Kate Longhurst,Left Defensive Midfield,Reading WFC,83.953439,84.782663,0.829224,1.009877,1.2,153.434949
3,7,Carry,True,71.2,74.1,77.4,74.1,1,0.0,Pass,Ball Receipt*,Ball Recovery,Pressure,NaN,1.742355,NaN,NaN,NaN,NaN,NaN,NaN,From Kick Off,Fara Williams,Center Attacking Midfield,Reading WFC,59.533604,54.567115,4.966489,0.916577,-6.2,0.000000
4,10,Carry,True,6.8,31.6,8.7,37.6,1,0.0,Pressure,Pass,Ball Receipt*,Pass,NaN,2.297598,NaN,NaN,NaN,NaN,NaN,NaN,From Free Kick,Anne Moorhouse,Goalkeeper,West Ham United LFC,113.511233,111.325873,2.185360,0.980748,-1.9,72.428741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28489,97735,Pass,True,65.1,9.1,69.6,8.1,1,0.0,Ball Receipt*,Interception,Pass,Carry,NaN,0.263238,-0.218669,Ground Pass,4.609772,Incomplete,Amalie Vevle Eikeland,NaN,From Throw In,Mayumi Pacheco,Left Back,Arsenal WFC,62.998571,59.647045,3.351526,0.946800,-4.5,347.471192
28490,97741,Carry,True,13.3,68.2,10.7,76.5,1,1.0,Pass,Ball Receipt*,Ball Recovery,Pressure,NaN,8.097732,NaN,NaN,NaN,NaN,NaN,NaN,Regular Play,Kim Little,Right Center Midfield,Reading WFC,110.363626,115.233415,4.869789,1.044125,2.6,107.393322
28491,97742,Pass,True,10.7,76.5,5.0,77.6,1,0.0,Ball Receipt*,Carry,Carry,Ball Recovery,NaN,1.068645,2.950954,Ground Pass,5.805170,NaN,Leah Williamson,NaN,Regular Play,Kim Little,Right Center Midfield,Reading WFC,115.233415,120.990743,5.757328,1.049962,5.7,169.077195
28492,97745,Carry,True,79.0,23.5,83.3,19.2,1,0.0,Pressure,Carry,Dribble,Dribbled Past,NaN,1.330094,NaN,NaN,NaN,NaN,NaN,NaN,Regular Play,Josanne Potter,Left Center Back,Reading WFC,44.195588,42.184476,2.011112,0.954495,-4.3,315.000000


# create categories basd on metrics to define how players behave under pressure

In [8]:
df1 = df
##create different categories of how players behave under pressure##
df1["presscarry"] = np.where((df1["type.name"]=="Carry") & (df1["prior_action_name"]=="Ball Receipt*") & (df1["nextactionu_p"]==0),1,0)
df1["pressmover"] = np.where((df1["nextactionu_p"]==1) & (df1["pass.outcome.name"]!="Incomplete"),1,0)
df1["pressreliever"] = np.where((df1["nextactionu_p"]==0) & (df1["changex"]>=-15) & (df1["changex"]<=15) & (df1["xstart"]<=80)& (df1["pass.outcome.name"]!="Incomplete") ,1,0)
df1["pressbreaker"] = np.where((df1["nextactionu_p"]==0) & (df1["changex"]<=-15)& (df1["pass.outcome.name"]!="Incomplete"),1,0)
df1["pressloser"] = np.where(df1["pass.outcome.name"]=="Incomplete",1,0)

In [9]:
## group players by categories and sort by pressbreaker ##
df3 = df1.groupby("player.name",sort=True,as_index=False)["pressmover","pressreliever","pressbreaker","pressloser","presscarry"].sum()
df3 = df3.sort_values(by=["pressbreaker"],ascending=False).reset_index(drop=True)
df3

C:\Users\philipp.hollenhorst\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,player.name,pressmover,pressreliever,pressbreaker,pressloser,presscarry
0,Leah Williamson,61,189,47,27,82
1,Stephanie Houghton,26,134,42,28,73
2,Melissa Lawley,53,133,39,18,100
3,Gabrielle George,41,128,37,24,71
4,Katie McCabe,58,136,36,32,97
5,Lauren Hemp,40,86,35,24,82
6,Niamh Charles,40,87,33,20,74
7,Rinsola Babajide,34,53,31,14,58
8,Felicity Gibbons,34,135,30,34,78
9,Janine Beckie,41,87,29,30,86


In [10]:
import json
import pandas as pd
from pandas import json_normalize
import numpy as np

import requests
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

from matplotlib import cm
import matplotlib.pyplot as plt
import cmasher as cmr
import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import matplotlib.cm
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.projections import get_projection_class
from matplotlib.patches import Arc
import io
import matplotlib.image as image
from matplotlib import transforms
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch

# create interactive visualizations

In [11]:
### create functions for pass and carry sonars ###

def sonar_pass1(p1,df,ax):
    pdf = df.loc[df["player.name"] == p1] ##filter only passes for the team in question
    pdf = pdf[pdf["type.name"]=="Pass"]
    df1 = pdf.loc[:, ["angle1","distance"]] ##just take in lengths and angles of passes
    bin_size=24
    bins = np.arange(0,360+bin_size,bin_size) ##change 24 to something else if you want more/less bins
    df1['bins'] = pd.cut(df1['angle1'], bins, include_lowest=True, right = True)
    df1["bin_mids"] = df1["bins"].apply(lambda x: x.mid)
    A = df1.groupby(["bin_mids"]).agg(frequency=("distance", "size"), alength = ("distance", "mean")).reset_index()
    A["freq"] = minmax_scale(A.frequency) ##scale to [0,1]
    A["freq"] = minmax_scale(A.frequency) ##scale to [0,1]
    A["centers"] = np.deg2rad(A.bin_mids.tolist())
    maxl = np.max(A["freq"])
    minl = np.min(A["freq"])
    normalizedplayer = [(x-minl)/(maxl-minl) for x in A["freq"]]
    ax.bar(A["centers"], A.alength, width=np.deg2rad(bin_size),
                    bottom=0.0,color="white", edgecolor='k',linewidth=1)
    ax.set_theta_zero_location("N")
    ax.set_theta_direction("clockwise")
    ax.set_ylim(0,25)
    ax.set_yticklabels([])
    ax.set_xticklabels(["forward","","right","","backwards","","left",""])
    ax.grid(False)
    ax.spines['polar'].set_color('w')

    ################################################
    pdf1 = df.loc[df["player.name"] == p1] ##filter only passes for the team in question
    pdf1 = pdf1[(pdf1["type.name"]=="Pass") & (pdf["pass.outcome.name"]!="Incomplete")]
    df11 = pdf1.loc[:, ["angle1","distance"]] ##just take in lengths and angles of passes
    bin_size=24
    bins = np.arange(0,360+bin_size,bin_size) ##change 24 to something else if you want more/less bins
    df11['bins'] = pd.cut(df11['angle1'], bins, include_lowest=True, right = True)
    df11["bin_mids"] = df11["bins"].apply(lambda x: x.mid)
    A1 = df11.groupby(["bin_mids"]).agg(frequency=("distance", "size"), alength = ("distance", "mean")).reset_index()
    A1["freq"] = minmax_scale(A1.frequency) ##scale to [0,1]
    A1["freq"] = minmax_scale(A1.frequency) ##scale to [0,1]
    A1["centers"] = np.deg2rad(A1.bin_mids.tolist())
    maxl = np.max(A1["freq"])
    minl = np.min(A1["freq"])
    normalizedplayer = [(x-minl)/(maxl-minl) for x in A1["freq"]]
    cmap = cmr.rainforest                   # CMasher
    cmap = plt.get_cmap('cmr.flamingo')
    ax.bar(A1["centers"], A1.alength, width=np.deg2rad(bin_size),
                    bottom=0.0,color=cmap(normalizedplayer), edgecolor='k',alpha=1,linewidth=2,linestyle="--")
    ax.set_theta_zero_location("N")
    ax.set_theta_direction("clockwise")
    ax.set_ylim(0,25)
    ax.set_yticklabels(["5 M", "10 M", "15 M", "20 M", "25 M"], color="black",fontweight="bold",rotation=180)
   
    ax.grid(False)
    ax.spines['polar'].set_color('w')
    
     
    
def sonar_carry1(p1,df,ax):
    pdf = df.loc[df["player.name"] == p1] ##filter only passes for the team in question
    pdf = pdf[pdf["type.name"]=="Carry"]
    df1 = pdf.loc[:, ["angle1","distance"]] ##just take in lengths and angles of passes
    bin_size=24
    bins = np.arange(0,360+bin_size,bin_size) ##change 24 to something else if you want more/less bins
    df1['bins'] = pd.cut(df1['angle1'], bins, include_lowest=True, right = True)
    df1["bin_mids"] = df1["bins"].apply(lambda x: x.mid)
    A = df1.groupby(["bin_mids"]).agg(frequency=("distance", "size"), alength = ("distance", "mean")).reset_index()
    A["freq"] = minmax_scale(A.frequency) ##scale to [0,1]
    A["freq"] = minmax_scale(A.frequency) ##scale to [0,1]
    A["centers"] = np.deg2rad(A.bin_mids.tolist())
    maxl = np.max(A["freq"])
    minl = np.min(A["freq"])
    normalizedplayer = [(x-minl)/(maxl-minl) for x in A["freq"]]
    cmap = cmr.rainforest                   # CMasher
    cmap = plt.get_cmap('cmr.flamingo')
    ax.bar(A["centers"], A.alength, width=np.deg2rad(bin_size),
                    bottom=0.0,color=cmap(normalizedplayer), edgecolor='k',alpha=1,linewidth=2,linestyle="--")
    ax.set_theta_zero_location("N")
    ax.set_theta_direction("clockwise")
    ax.set_ylim(0,10)
    ax.set_yticklabels(["2","4","6","8","10"],color="black",fontweight="bold",rotation=180)
    ax.set_xticklabels(["forward","","right","","backwards","","left",""])
    ax.grid(False)
    ax.spines['polar'].set_color('w')

In [17]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output

xr1=widgets.IntRangeSlider(value=[0, 120],min=0,max=120,description='X Range:',
                                        step=1,disabled=False,style={'description_width': 'initial'},
                                        readout=True,readout_format='.1f')
yr1=widgets.IntRangeSlider(value=[0, 68],min=0,max=80,description='Y Range:',
                                        step=1,disabled=False,style={'description_width': 'initial'},
                                        readout=True,readout_format='.1f')

ang1=widgets.FloatSlider(value=360,min=0,max=360.0,
                            step=1,description='Angular Range:',
                         style={'description_width': 'initial'},
                        disabled=False,continuous_update=False,
                        orientation='horizontal',readout=True,
                        readout_format='.1f')

player1 = widgets.Dropdown(
    options=df["player.name"].unique(),
    value='Leah Williamson',
    description='Player 1',
    disabled=False,
)


pitch =VerticalPitch(pitch_type='statsbomb',line_zorder=2,
                      line_color='#636363',constrained_layout=True,
                      tight_layout=False,pad_top=10)
#presstype = widgets.Dropdown(
#    options=["up,pressbreaker]
ui = widgets.HBox([xr1,yr1, ang1, player1])

def arrowplotter(dataframe,pitch,ax,color,X1,Y1,X2,Y2,zorder):
    pitch.arrows(dataframe[X1], dataframe[Y1],
             dataframe[X2], dataframe[Y2], width=2, zorder= zorder,
             headwidth=3, headlength=3, color=color, ax=ax)

    pitch =VerticalPitch(pitch_type='statsbomb',line_zorder=2,
                      line_color='#636363',constrained_layout=True,
                      tight_layout=False,pad_top=10)

    

def Pressureplotter(xrange,yrange,angle,p1):

    fig = plt.figure(figsize=(15, 15))
    # fig.subplots_adjust(wspace=0)
    ax1 = fig.add_subplot(221)
    ax = fig.add_subplot(222, polar=True)
    dff = df[df["player.name"]==p1]
    dffp = dff[dff["type.name"]=="Pass"]
    fig.set_facecolor('w')
    pitch.draw(ax1)
    cmap = cmr.flamingo                   # CMasher
    cmap = plt.get_cmap('cmr.flamingo')
    
    cond1 = ((dffp['angle1'] > 0.0) & (dffp['angle1'] <= angle))
    cond2 = ((dffp['angle1'] >= 360-angle) & (dffp['angle1'] <= 360.0))
    zonex = (dffp.xend >= xrange[0]) & (dffp.xend <= xrange[1])
    zoney = (dffp.yend >= yrange[0]) & (dffp.yend <= yrange[1])
    
    playername1 = dffp["player.name"] ==p1
    
    arrowplotter(dffp[(cond1 | cond2) & playername1 & zonex & zoney],pitch,ax1,
                 'pink','xstart','ystart','xend','yend',4)
    
    ax1.title.set(text=p1 + " Under Pressure Passes", color="k", fontweight="bold", size=15)
    
    sonar_pass1(p1,dffp[(cond1 | cond2) & playername1 & zonex & zoney],ax)
    ax.title.set(text=p1 + " Under Pressure Pass Sonar", color="k", fontweight="bold", size=15)
    
    return fig
   


w2 = interactive_output(Pressureplotter,{'xrange':xr1,'yrange':yr1,'angle':ang1,'p1':player1})
display(ui, w2)

Output()

In [18]:
def Pressureplotter_Carry(xrange,yrange,angle,p1):

    fig = plt.figure(figsize=(15, 15))
    # fig.subplots_adjust(wspace=0)
    ax1 = fig.add_subplot(221)
    ax = fig.add_subplot(222, polar=True)
    dff = df[df["player.name"]==p1]
    dffp = dff[dff["type.name"]=="Carry"]
    fig.set_facecolor('w')
    pitch.draw(ax1)
    cmap = cmr.flamingo                   # CMasher
    cmap = plt.get_cmap('cmr.flamingo')
    
    cond1 = ((dffp['angle1'] > 0.0) & (dffp['angle1'] <= angle))
    cond2 = ((dffp['angle1'] >= 360-angle) & (dffp['angle1'] <= 360.0))
    zonex = (dffp.xend >= xrange[0]) & (dffp.xend <= xrange[1])
    zoney = (dffp.yend >= yrange[0]) & (dffp.yend <= yrange[1])
    
    
    playername1 = dffp["player.name"] ==p1
    
    arrowplotter(dffp[(cond1 | cond2) & playername1 & zonex & zoney],pitch,ax1,
                 'pink','xstart','ystart','xend','yend',4)
    
    ax1.title.set(text="Carry Map", color="k", fontweight="bold", size=15)
    
    sonar_carry1(p1,dffp[(cond1 | cond2) & playername1 & zonex & zoney],ax)
    ax.title.set(text="Carry Sonar", color="k", fontweight="bold", size=15)
    #fig.suptitle(p1 + ' Carries under Pressure',fontweigt="bold" fontsize=16)
    
    return fig
   


w2 = interactive_output(Pressureplotter_Carry,{'xrange':xr1,'yrange':yr1,'angle':ang1,'p1':player1})
display(ui, w2)




Output()